In [1]:
import os
import re
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

In [2]:
## Lemmatizations
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\Lakshmi
[nltk_data]     Praffulla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Lakshmi
[nltk_data]     Praffulla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
## Opening the files

directory_r = [f for f in os.listdir('./20_newsgroups') if not f.startswith('.')]
directory_r

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [7]:
favorable_reviews = []
with open('file.txt') as f:
    lines = f.readlines()
    
for i in lines:
    i = i.replace("\n","")
    k = i.split(" ")
    favorable_reviews.append(int(k[1]))

In [8]:
len(favorable_reviews)

19997

In [9]:
def lematization(data):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for i in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(i)
    
    return new_text

In [10]:
def convert_lower_case(data):
    return np.char.lower(data)

In [11]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [12]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [13]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [14]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [15]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [16]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) 
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    
    data = lematization(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [209]:
processed_text = []
path_id = []
final_path = []
co = 0

for i in range(len(directory_r)):
    files = os.listdir('./20_newsgroups/' + directory_r[i])
    for j in range(len(files)):
        path = './20_newsgroups/' + directory_r[i] + '/' + files[j]
        doc_id = files[j]
        path_id.append(doc_id)
        final_path.append(path)
        text = open(path,'r',errors='ignore').read()
        
        processed_text.append(word_tokenize(str(preprocess(text))))
        
        
        if co % 500 == 0:
            print(co)
        co += 1

In [18]:
len(processed_text)

19997

In [19]:
# processed_text has all the text_data of each document stored in a list
# processed_text = [[Doc_0 text],[Doc_1 text],.....]

#Natural Term Frequency = tf

natural_term_frequency = []
for item in processed_text:
    dictionary_tf ={}
    
    for word in item:
        if dictionary_tf.get(word) == None:
            dictionary_tf[word] = 1
        else:
            count = dictionary_tf.get(word)
            count += 1
            dictionary_tf[word] = count
    
    natural_term_frequency.append(dictionary_tf)
        

In [21]:
# Logarithmic Term Frequency = 1 + log(tf)

logarithmic_term_frequency = []

for item in natural_term_frequency:
    dictionary_ltf = {}
    allkeys = item.keys()
    
    for word in allkeys:
        natural_freq = item.get(word)
        log_freq = 1 + np.log(natural_freq)
        dictionary_ltf[word] = log_freq
        
    logarithmic_term_frequency.append(dictionary_ltf)

In [22]:
# logarithmic_term_frequency

In [23]:
#Document Frequency 

document_frequency = {}

for item in processed_text:
    
    unique_terms = list(set(item))
    
    for word in unique_terms:
        
        if document_frequency.get(word) == None:
            document_frequency[word] = 1
        else:
            count = document_frequency.get(word)
            count += 1
            document_frequency[word] = count

In [24]:
len(document_frequency)

164428

In [25]:
print(len(document_frequency))
N = len(processed_text)
print('Total Number of documents are',N)

164428
Total Number of documents are 19997


In [26]:
#Inverse Document Frequency 

inverse_document_frequency = {}

keys_df = document_frequency.keys()

for item in keys_df:
    
    getvalue = document_frequency[item]
    in_doc_freq = np.log(N/getvalue)
    inverse_document_frequency[item] = in_doc_freq


In [27]:
#TF-IDF weighting 
#Natural_Term_Frequency

list_return = []

for item in natural_term_frequency:
    dict_inside = {}
    allkeys = item.keys()
    
    for word in allkeys:
        current_freq = item[word]
        idf_for_word = inverse_document_frequency[word]
        update_freq = current_freq * idf_for_word
        dict_inside[word] = update_freq
    
    list_return.append(dict_inside)


In [30]:
# list_return

In [32]:
#Function block for calculating TF-IDF


def calculate_tfidf(list_tf):
    list_return = []
    co = 0
    for item in list_tf:
        dict_inside = {}
        allkeys = item.keys()
        
        for word in allkeys:
            current_freq = item[word]
            idf_for_word = inverse_document_frequency[word]
            update_freq = current_freq * idf_for_word
            update_freq = update_freq + favorable_reviews[co]
            dict_inside[word] = update_freq
    
        list_return.append(dict_inside)
        co += 1
    return list_return

In [33]:
# logarithmic_term_frequency

In [42]:
tfidf_log = calculate_tfidf(logarithmic_term_frequency)

In [43]:
# tfidf_log

In [46]:
len(document_frequency)

164428

In [1]:
inverted_index = {}
co = 0
for word in document_frequency:
    dict_toappend = {}
    index = 0
    for doc in tfidf_log:
        if word in doc:
            dict_toappend[path_id[index]] = doc[word]
        index += 1    
    inverted_index[word] = dict_toappend
    if co % 5000 == 0:
        print(co)
    co += 1    

In [2]:
inverted_index

In [69]:
import operator

In [70]:
sorted_inverted_index = {} 
for item in inverted_index:
    getdict = inverted_index[item]
    getdictappend = dict(sorted(getdict.items(),key=operator.itemgetter(1),reverse=True))
    sorted_inverted_index[item] = getdictappend

In [188]:
# sorted_inverted_index

In [72]:
campionlist_word = {}
for item in sorted_inverted_index:
    campionlist_word[item] = list(sorted_inverted_index[item].keys())    

In [187]:
# campionlist_word

In [79]:
## Now we will choose "R", which is the size of the chanpion list
## we will break them into two list

In [89]:
r = 20

In [90]:
highlow_camp = {} 
for item in campionlist_word: 
    getlist = campionlist_word[item] 
    highlow_camp[item] = [getlist[:r],getlist[r:]] 

In [110]:
highlow_camp['canyon'][0][:1] 

['105563']

In [172]:
def getqueryvector(query):
      
    query_text = word_tokenize(str(preprocess(query)))
    
    #term_freq_query
    term_freq_query = {}

    for item in query_text:
    
        if item in term_freq_query:
            temp = term_freq_query[item]
            temp += 1
            term_freq_query[item] = temp
        else:
             term_freq_query[item] = 1
    
    
    # Coverting into log frequency

    for item in term_freq_query.keys():
    
        temp = term_freq_query[item]
        new = 1 + np.log(temp)
        term_freq_query[item] = new
    
    #inverse_document_frequency
    tf_idf_query = {}

    for item in term_freq_query.keys():
    
        if item in inverse_document_frequency :
        
            temp = inverse_document_frequency[item]
            toadd = temp * term_freq_query[item]
        
            tf_idf_query[item] = toadd
        
        else:
            tf_idf_query[item] = 0.0
    
    
    #  query vector 

    query_vector = []
#     list(document_frequency.keys())
    for word in query_text:
    
        if word in tf_idf_query :
            temp = tf_idf_query[word]
            query_vector.append(temp)
        else:
            query_vector.append(0.0)
            
    #Normalization of query vector    
    
    norm_query_vector = []

    normal = 0.0
    for item in query_vector:
        normal += (item*item)
    
    normal = 1/np.sqrt(normal)
    for item in query_vector:
        norm_query_vector.append(item * normal)
    
    
    return norm_query_vector

In [203]:
k = 10
query = ""
query_text = word_tokenize(str(preprocess(query)))
queryvector = getqueryvector(query)

In [177]:
documents_eval = []
if k < r:
    # we will implement high list  
    for item in highlow_camp:
        documents_eval += highlow_camp[item][0][:k]    
else:
    # we should also consider low list
    extra = r - k
    for item in highlow_camp:
        documents_eval += highlow_camp[item][0]
        documents_eval += highlow_camp[item][1][:extra]
        
documents_eval = list(set(documents_eval))        

In [178]:
len(documents_eval)

16299

In [184]:
## For each document we have to find out vector

document_vector = []

for eachdoc in documents_eval:
    getplace = path_id.index(eachdoc)
    getdict = tfidf_log[getplace]
    
    inside_list = []
    
    for word in query_text:
        if word in getdict:
            temp = getdict[word]
            inside_list.append(temp)
        else:
            inside_list.append(0.0)
    document_vector.append(inside_list)    

In [186]:
# document_vector

In [182]:
## Normalize the document vector

norm_doc_vector = []

for item in document_vector:
    
    norm = 0.0
    for freq in item:
        norm += (freq*freq)
        
    if np.sqrt(norm) == 0.0 :
        norm == 0.0
    else:
        norm = 1/np.sqrt(norm)
    
    temp_list = []
    
    for freq in item :
        k = freq * norm
        temp_list.append(k)
    norm_doc_vector.append(temp_list)

In [190]:
cosine_scores = {}

doc_id = -1
for item in norm_doc_vector:
    
    doc_id += 1
    list_doc = np.array(item)
    list_query = np.array(queryvector)
    score = sum(list(list_doc * list_query))
    
    cosine_scores[doc_id] = score
    
sorted_cosine_scores = dict(sorted(cosine_scores.items(),key=operator.itemgetter(1),reverse=True))

In [198]:
# sorted_cosine_scores

In [213]:
anslist = list(sorted_cosine_scores.keys())[:k]
print("The top ",k,"Documenst for the query is:",anslist)
print
for i in anslist:
    print(final_path[i])

The top  10 Documenst for the query is: [4314, 10571, 8244, 1246, 7973, 13692, 1156, 12981, 12968, 8574]
./20_newsgroups/comp.sys.mac.hardware/51673
./20_newsgroups/rec.sport.hockey/53971
./20_newsgroups/rec.motorcycles/104393
./20_newsgroups/comp.graphics/38407
./20_newsgroups/rec.autos/103771
./20_newsgroups/sci.med/59332
./20_newsgroups/comp.graphics/38317
./20_newsgroups/sci.electronics/54485
./20_newsgroups/sci.electronics/54351
./20_newsgroups/rec.motorcycles/104724
